Batch Adding Users from a CSV file.
Let's first confirm who the existing users are:

In [1]:
import os
os.chdir("..")
from config import *

# Function to fetch and print existing users with user ID, email, and last updated
def get_existing_users():
    response = requests.get(f"{base_url}/users?limit=9999", headers=headers_get)
    if response.status_code == 200:
        users = response.json().get('data', [])
        if not users:
            print("No users found.")
            return {}

        user_dict = {}
        for user in users:
            user_id = user.get('id', 'N/A')
            email = user.get('email', 'N/A')
            updated_at = user.get('updatedAt', None)

            # Convert updatedAt (if exists) to human-readable format
            if updated_at:
                updated_at = datetime.fromtimestamp(updated_at / 1000).strftime('%Y-%m-%d %H:%M:%S')
            else:
                updated_at = 'Unknown'

            print(f"User ID: {user_id}, Last Updated: {updated_at}, Email: {email}")
            user_dict[email] = {'id': user_id, 'last_updated': updated_at}
        
        return user_dict
    else:
        print(f"Failed to fetch users: {response.status_code}")
        return {}

# Get the full user dictionary
users_dict = get_existing_users()

# Extract emails as a list
email_list = list(users_dict.keys())

print(f'\nThere are a total of {len(email_list)} users at present.')

User ID: 90efee84-885b-4d2b-b27c-95f4a136414a, Last Updated: 2024-11-22 17:21:00, Email: abigail.michel@gov.bc.ca
User ID: c32c5a3c-2b43-4b03-abe5-e97aff610c22, Last Updated: 2024-11-27 16:18:54, Email: Adam.Dewey@gov.bc.ca
User ID: 51d3ebad-7eeb-407c-855f-264530b693d0, Last Updated: 2024-11-27 16:41:33, Email: Adria.Fradley@gov.bc.ca
User ID: 1899f8ce-7086-482b-8d3c-51f890a43a85, Last Updated: 2024-11-27 16:48:05, Email: adrienne.canty@gov.bc.ca
User ID: c3bec711-761b-460c-a148-cfd7fe4098e2, Last Updated: 2024-11-27 16:43:23, Email: Albert.Law@gov.bc.ca
User ID: b47fb8c3-5984-44da-a0f3-cb9ef4ffc6e9, Last Updated: 2024-11-27 16:39:16, Email: Albert.Nussbaum@gov.bc.ca
User ID: 72b459dd-490a-41a4-bb7f-fec6e618ca6d, Last Updated: 2024-11-27 16:41:21, Email: Alec.Dale@gov.bc.ca
User ID: 17f22740-bc09-4f48-a16a-cb95be19ff61, Last Updated: 2024-11-25 07:44:42, Email: alex.nishta@gov.bc.ca
User ID: 03435607-5b8a-4194-8508-e0aa0e0ff461, Last Updated: 2024-11-27 16:48:19, Email: Allan.Lidstone@

In [10]:
users_dict['David.Tesch@gov.bc.ca']['id']

'87d5a3d0-7957-416f-86b4-ee43ef55ebf9'

In [33]:
# Go ahead and import the CSV data and get the User ID of these custodians
df_users = pd.read_csv('reference_csvs/nov2024_custodians.csv', header=1)

# Assuming `users_dict` is a dictionary where keys are emails and values are user data
df_users['user_ID'] = df_users['Custodian 2'].apply(lambda x: users_dict.get(x, {}).get('id', 'Not Found'))

df_users.loc[1, 'user_ID'] = '87d5a3d0-7957-416f-86b4-ee43ef55ebf9' # Manually fixing a bug with one user

# Preview the resulting DataFrame
df_users.head()

,Application,Status,Custodian 1,Custodian 2,user_ID
0,AACDET,Retired,"Nussbaum, Albert",Albert.Nussbaum@gov.bc.ca,b47fb8c3-5984-44da-a0f3-cb9ef4ffc6e9
1,ACAT,Active,"Tesch, David",David.Tesch@gov.bc.ca,87d5a3d0-7957-416f-86b4-ee43ef55ebf9
2,ACRFD,Active,"Yetskalo, Valentina",Valentina.Yetskalo@gov.bc.ca,54ebfcc9-90e9-4916-bbd5-b1a27557a49e
3,ACS,Active,"Searle, Barb",Barb.Searle@gov.bc.ca,c1ef187e-4980-48a2-a294-b08988db4eab
4,ADAM (FOR),Active,"Dewey, Adam",Adam.Dewey@gov.bc.ca,c32c5a3c-2b43-4b03-abe5-e97aff610c22
